In [1]:
#install spacy sentence segmentizer
!pip install spacyss

  Created wheel for spacyss: filename=spacyss-1.0.1-cp36-none-any.whl size=2781 sha256=3134f0c9d7fe75dce054902ed8dc4b8363a6d92587488dd168d1313c101c7641
  Stored in directory: /root/.cache/pip/wheels/bc/ee/17/12d7d797bcb813a0e180fe2951222acb115f803778fe79afef
Successfully built spacyss


In [2]:
pip install nlp

     |████████████████████████████████| 1.7MB 5.6MB/s 
     |████████████████████████████████| 17.7MB 195kB/s 
     |████████████████████████████████| 245kB 36.5MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
pip install datasets

     |████████████████████████████████| 153kB 4.3MB/s 


In [4]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 51798 (delta 10), reused 4 (delta 0), pack-reused 51769
Receiving objects: 100% (51798/51798), 38.39 MiB | 24.04 MiB/s, done.
Resolving deltas: 100% (36246/36246), done.


In [5]:
cd transformers

/content/transformers


In [6]:
pip install .

Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 4.3MB/s 
     |████████████████████████████████| 890kB 33.6MB/s 
  Created wheel for transformers: filename=transformers-4.0.0.dev0-cp36-none-any.whl size=1345137 sha256=c32cb1986f3347e442b2f80ae738a5b841c221c02e81727bb44c6f80aa0e3527
  Stored in directory: /tmp/pip-ephem-wheel-cache-4flx9wa8/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=24f3a83b1d1594ad29066dbd5ed0c954d5762811f4a32cbeb4e3e67ac7abeb4f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
cd /content

/content


In [8]:
#Download SQuAD 2.0 Dataset for train and dev
!wget -O train-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -O dev-v2.0.json https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-11-18 15:43:28--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  63.5MB/s    in 0.6s    

2020-11-18 15:43:29 (63.5 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-11-18 15:43:29--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [9]:
#load dataset
import json

with open('/content/train-v2.0.json') as d:
    train = json.load(d)

with open("/content/dev-v2.0.json") as t:
    dev = json.load(t)

In [ ]:
# import pandas as pd

# train_df = pd.read_json('/content/train-v2.0.json')
# test_df = pd.read_json('/content/dev-v2.0.json')

In [10]:
train1 = dict(list(train.items())[len(train)//2:])
train2 = dict(list(train.items())[:len(train)//2])

dev1 = dict(list(dev.items())[len(dev)//2:])
dev2 = dict(list(dev.items())[len(dev)//2:])

In [11]:
#import spacyss and set
from spacy.lang.en import English
nlp = nlp = English()
sbd = nlp.create_pipe('sentencizer')
nlp.add_pipe(sbd)

In [12]:
#make dict {Question: Sent /w Ans} for train Dataset
train_inputs= ()

for p in range(len(train1["data"])):   
    for q in range(len(train1["data"][p]["paragraphs"])):
        context = train1["data"][p]["paragraphs"][q]["context"]
        doc = nlp(context)
        sens = []
        for sent in doc.sents:
            sens.append(sent.text)
        for r in range(len(train1["data"][p]["paragraphs"][q]["qas"])):
            question = train1["data"][p]["paragraphs"][q]["qas"][r]["question"]
            if train1["data"][p]["paragraphs"][q]["qas"][r]["is_impossible"] == False:
                #answer = train["data"][p]["paragraphs"][q]["qas"][r]["answers"][0]["text"]
                answer_start = train1["data"][p]["paragraphs"][q]["qas"][r]["answers"][0]["answer_start"]
                count = 0
                for i in range(len(sens)):
                    if count < answer_start <= (count + len(sens[i])):
                        temp = list(train_inputs)
                        temp.append([question, sens[i]])
                        train_inputs = tuple(temp)
                        break
                    else:
                        count += len(sens[i]) + 1
            #else:
                #temp = list(train_inputs)
                #temp.append([question, ""])
                #train_inputs = tuple(temp)
#print(train_inputs) #Error 'IOPub data rate exceeded' Occurs

In [13]:
#make dict {Question: Sent /w Ans} for dev Dataset
dev_inputs= ()

for p in range(len(dev1["data"])):   
    for q in range(len(dev1["data"][p]["paragraphs"])):
        context = dev1["data"][p]["paragraphs"][q]["context"]
        doc = nlp(context)#
        sens = []
        for sent in doc.sents:
            sens.append(sent.text)
        for r in range(len(dev1["data"][p]["paragraphs"][q]["qas"])):
            question = dev1["data"][p]["paragraphs"][q]["qas"][r]["question"]
            if dev1["data"][p]["paragraphs"][q]["qas"][r]["is_impossible"] == False:
                #answer = train["data"][p]["paragraphs"][q]["qas"][r]["answers"][0]["text"]
                answer_start = dev1["data"][p]["paragraphs"][q]["qas"][r]["answers"][0]["answer_start"]
                count = 0
                for i in range(len(sens)):
                    if count < answer_start <= (count + len(sens[i])):
                        temp = list(dev_inputs)
                        temp.append([question, sens[i]])
                        dev_inputs = tuple(temp)
                        break
                    else:
                        count += len(sens[i]) + 1
            #else:
                #temp = list(dev_inputs)
                #temp.append([question, ""])
                #dev_inputs = tuple(temp)
print(dev_inputs)

(['In what country is Normandy located?', 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.'], ['When were the Normans in Normandy?', 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.'], ['From which countries did the Norse originate?', 'They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.'], ['Who was the Norse leader?', 'They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia.'], ['What century did the Normans first gain their separate identity?', 'The

In [14]:
f = open('input.txt', 'w')
i = 0
for d in train_inputs:
    if d[1] != "":
        if i != len(dev_inputs)-1:
            string = d[1] + d[0] + "<|endoftext|>" + "\n"
            f.write(string)
            i += 1
        else:
            string = d[1] + d[0] + "<|endoftext|>"
            f.write(string)
f.close()

i = 0
f = open('dev.txt', 'w')
for d in dev_inputs:
    if d[1] != "":
        if i != len(dev_inputs)-1:
            string = d[1] + d[0] + "<|endoftext|>" + "\n"
            f.write(string)
            i += 1
        else:
            string = d[1] + d[0] + "<|endoftext|>"
            f.write(string)
f.close()

i = 0
f = open('pred.txt', 'w')
for d in dev_inputs:
    if d[1] != "":
        if i != len(dev_inputs)-1:
            string = d[1] + "^" + d[0]  + "\n"
            f.write(string)
            i += 1
        else:
            string = d[1] + "^" + d[0]
            f.write(string)
f.close()

In [ ]:
#pickle로 저장
#import pickle
# with open('train_data.pickle', 'wb') as f:
#     pickle.dump(train_inputs, f, pickle.HIGHEST_PROTOCOL)

# with open('dev_data.pickle', 'wb') as f:
#     pickle.dump(dev_inputs, f, pickle.HIGHEST_PROTOCOL)


In [ ]:
#json으로 저장
# import json

# with open('train_data.json', 'w', encoding='utf-8') as tj:
#     json.dump(train_inputs, tj, indent="\t")

# with open('dev_data.json', 'w', encoding='utf-8') as dj:
#     json.dump(dev_inputs, dj, indent="\t")


In [16]:
!python /content/transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --model_type=gpt2 \
    --do_train \
    --train_file=/content/input.txt \
    --do_eval \
    --validation_file=/content/dev.txt \
    --save_steps=2000 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --num_train_epochs=3 \
    --overwrite_output_dir \
    --output_dir=/content/drive/My\ Drive/Yonsei/CS4-2/Sojong20Fall/test-clm2

2020-11-18 15:46:45.863260: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/18/2020 15:46:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
11/18/2020 15:46:49 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/drive/My Drive/Yonsei/CS4-2/Sojong20Fall/test-clm2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Nov18_15-46-48_5203c9419dde', l

In [ ]:
pip install nltk --upgrade

     |████████████████████████████████| 1.4MB 13.8MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=676ddf1288027c478c3f1683a9c55b23c62f1416c62a3e63f0c32c53d43f12ca
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!python /content/transformers/examples/text-generation/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=/content/drive/My\ Drive/Yonsei/CS4-2/Sojong20Fall/test-clm \
    --num_return_sequences=1 \
    --input_file=/content/pred.txt \
    --length=50

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
=== GENERATED SEQUENCE 1 ===
What ability Madonna has had in the past have grown in their work?
9233
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Besides singing Madonna has the ability to play several musical instruments.Which singer has the ability to play a variety of musical instruments?
9234
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
What instrument plays at the top of Madonna's solo repertoire?
9235
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
What has Madonna learned as a songwriter?
9236
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
What type of vocal gymnastics is Madonna?
9237
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
How did Madonna create her singing voice?
92

/content/transformers/examples/text-generation/run_generation.py 에 덮어 쓰기

In [ ]:
#/content/transformers/examples/text-generation/run_generation.py

#!/usr/bin/env python3
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Conditional text generation with the auto-regressive models of the library (GPT/GPT-2/CTRL/Transformer-XL/XLNet)
"""


from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

import argparse
import logging

import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)


logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

# Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PREFIX = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


#
# Functions to prepare models' input
#


def prepare_ctrl_input(args, _, tokenizer, prompt_text):
    if args.temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(args, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if args.xlm_language in available_languages:
            language = args.xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


def prepare_transfoxl_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_type",
        default='gpt2',
        type=str,
        required=True,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path",
        default="/content/drive/My Drive/Yonsei/CS4-2/Sojong20Fall/test-clm" ,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--input_file",
        default='/content/pred.txt',
        type=str,
        required=True,
    )

    parser.add_argument("--prompt", type=str, default="")
    parser.add_argument("--length", type=int, default=50)
    parser.add_argument("--stop_token", type=str, default='|<endoftext>|', help="Token at which text generation is stopped")

    parser.add_argument(
        "--temperature",
        type=float,
        default=1.0,
        help="temperature of 1.0 has no effect, lower tend toward greedy sampling",
    )
    parser.add_argument(
        "--repetition_penalty", type=float, default=1.0, help="primarily useful for CTRL model; in that case, use 1.2"
    )
    parser.add_argument("--k", type=int, default=0)
    parser.add_argument("--p", type=float, default=0.9)

    parser.add_argument("--prefix", type=str, default="", help="Text added prior to input.")
    parser.add_argument("--padding_text", type=str, default="", help="Deprecated, the use of `--prefix` is preferred.")
    parser.add_argument("--xlm_language", type=str, default="", help="Optional language when used with the XLM model.")

    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument("--num_return_sequences", type=int, default=1, help="The number of samples to generate.")
    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    args = parser.parse_args()

    args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()

    logger.warning(
        "device: %s, n_gpu: %s, 16-bits training: %s",
        args.device,
        args.n_gpu,
        args.fp16,
    )

    set_seed(args)

    # Initialize the model and tokenizer
    try:
        args.model_type = args.model_type.lower()
        model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    except KeyError:
        raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
    model = model_class.from_pretrained(args.model_name_or_path)
    model.to(args.device)

    if args.fp16:
        model.half()

    args.length = adjust_length_to_model(args.length, max_sequence_length=model.config.max_position_embeddings)
    logger.info(args)
	
    f = open(args.input_file, 'r')
    num = 0
    summ = [0, 0, 0, 0]
    f1 = open('/content/gens.txt', 'w')
    lines = f.readlines()
    for i in range(len(lines)):
        try:
            prompt_text = lines[i]

            #prompt_text = args.prompt if args.prompt else input("Model prompt >>> ")

            # Different models need different input formatting and/or extra arguments
            requires_preprocessing = args.model_type in PREPROCESSING_FUNCTIONS.keys()
            if requires_preprocessing:
                prepare_input = PREPROCESSING_FUNCTIONS.get(args.model_type)
                preprocessed_prompt_text = prepare_input(args, model, tokenizer, prompt_text)

                if model.__class__.__name__ in ["TransfoXLLMHeadModel"]:
                    tokenizer_kwargs = {"add_space_before_punct_symbol": True}
                else:
                    tokenizer_kwargs = {}

                encoded_prompt = tokenizer.encode(
                    preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", **tokenizer_kwargs
                )
            else:
                prefix = args.prefix if args.prefix else args.padding_text
                encoded_prompt = tokenizer.encode(prefix + prompt_text, add_special_tokens=False, return_tensors="pt")
            encoded_prompt = encoded_prompt.to(args.device)

            if encoded_prompt.size()[-1] == 0:
                input_ids = None
            else:
                input_ids = encoded_prompt

            output_sequences = model.generate(
                input_ids=input_ids,
                max_length=args.length + len(encoded_prompt[0]),
                temperature=args.temperature,
                top_k=args.k,
                top_p=args.p,
                repetition_penalty=args.repetition_penalty,
                do_sample=True,
                num_return_sequences=args.num_return_sequences,
            )

            # Remove the batch dimension when returning multiple sequences
            if len(output_sequences.shape) > 2:
                output_sequences.squeeze_()

            generated_sequences = []

            for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
                print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
                generated_sequence = generated_sequence.tolist()

                # Decode text
                text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

                # Remove all text after the stop token
                text = text[: text.find(args.stop_token) if args.stop_token else None]

                # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
                total_sequence = (
                    text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]#prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
                )

                generated_sequences.append(total_sequence)
                #print(prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
                print(total_sequence)
                #print(reference)

                #references = [reference.split(" ")]
                #hypotheses = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :].split(" ")
                #references = " " + references
                #references = [tokenizer.encode(references)]
                #hypotheses = tokenizer.encode(text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
                #bleus = [sentence_bleu(references, hypotheses, weights=(1, 0, 0, 0)), sentence_bleu(references, hypotheses, weights=(0, 1, 0, 0)), sentence_bleu(references, hypotheses, weights=(0, 0, 1, 0)), sentence_bleu(references, hypotheses, weights=(0, 0, 0, 1))]
                #print("bleu:", bleus)
                #for g in range(len(bleus)):
                #    sum[g] += bleus[g]
                #num += 1
                if i == len(lines)-1:
                    f.write(total_sequence)
                else:
                    f.write(total_sequence) + "\n"
        except:
            pass

    #for g in range(len(bleus)):
    #    summ[g] = summ[g] / len(lines)
    #print(summ)
    f1.close()
    f.close()
    return generated_sequences


if __name__ == "__main__":
    main()

In [ ]:
#/content/transformers/examples/text-generation/run_generation.py

#!/usr/bin/env python3
# coding=utf-8
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Conditional text generation with the auto-regressive models of the library (GPT/GPT-2/CTRL/Transformer-XL/XLNet)
"""


from nltk.translate.bleu_score import sentence_bleu
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

import argparse
import logging

import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)


logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

# Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PREFIX = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


#
# Functions to prepare models' input
#


def prepare_ctrl_input(args, _, tokenizer, prompt_text):
    if args.temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(args, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if args.xlm_language in available_languages:
            language = args.xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


def prepare_transfoxl_input(args, _, tokenizer, prompt_text):
    prefix = args.prefix if args.prefix else args.padding_text if args.padding_text else PREFIX
    prompt_text = prefix + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": prepare_ctrl_input,
    "xlm": prepare_xlm_input,
    "xlnet": prepare_xlnet_input,
    "transfo-xl": prepare_transfoxl_input,
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_type",
        default='gpt2',
        type=str,
        required=True,
        help="Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--model_name_or_path",
        default="/content/drive/My Drive/Yonsei/CS4-2/Sojong20Fall/test-clm" ,
        type=str,
        required=True,
        help="Path to pre-trained model or shortcut name selected in the list: " + ", ".join(MODEL_CLASSES.keys()),
    )
    parser.add_argument(
        "--input_file",
        default='/content/pred.txt',
        type=str,
        required=True,
    )

    parser.add_argument("--prompt", type=str, default="")
    parser.add_argument("--length", type=int, default=50)
    parser.add_argument("--stop_token", type=str, default='|<endoftext>|', help="Token at which text generation is stopped")

    parser.add_argument(
        "--temperature",
        type=float,
        default=1.0,
        help="temperature of 1.0 has no effect, lower tend toward greedy sampling",
    )
    parser.add_argument(
        "--repetition_penalty", type=float, default=1.0, help="primarily useful for CTRL model; in that case, use 1.2"
    )
    parser.add_argument("--k", type=int, default=0)
    parser.add_argument("--p", type=float, default=0.9)

    parser.add_argument("--prefix", type=str, default="", help="Text added prior to input.")
    parser.add_argument("--padding_text", type=str, default="", help="Deprecated, the use of `--prefix` is preferred.")
    parser.add_argument("--xlm_language", type=str, default="", help="Optional language when used with the XLM model.")

    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization")
    parser.add_argument("--no_cuda", action="store_true", help="Avoid using CUDA when available")
    parser.add_argument("--num_return_sequences", type=int, default=1, help="The number of samples to generate.")
    parser.add_argument(
        "--fp16",
        action="store_true",
        help="Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit",
    )
    args = parser.parse_args()

    args.device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = 0 if args.no_cuda else torch.cuda.device_count()

    logger.warning(
        "device: %s, n_gpu: %s, 16-bits training: %s",
        args.device,
        args.n_gpu,
        args.fp16,
    )

    set_seed(args)

    # Initialize the model and tokenizer
    try:
        args.model_type = args.model_type.lower()
        model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
    except KeyError:
        raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

    tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path)
    model = model_class.from_pretrained(args.model_name_or_path)
    model.to(args.device)

    if args.fp16:
        model.half()

    args.length = adjust_length_to_model(args.length, max_sequence_length=model.config.max_position_embeddings)
    logger.info(args)
	
    f = open(args.input_file, 'r')
    num = 0
    summ = [0, 0, 0, 0]
    f = open('/content/gens.txt', 'w')
    lines = f.readlines()
    for i in range(len(lines)):
        try:
            temp = lines[i].split('^')
            prompt_text = temp[0]
            reference = temp[1].rstrip()

            #prompt_text = args.prompt if args.prompt else input("Model prompt >>> ")

            # Different models need different input formatting and/or extra arguments
            requires_preprocessing = args.model_type in PREPROCESSING_FUNCTIONS.keys()
            if requires_preprocessing:
                prepare_input = PREPROCESSING_FUNCTIONS.get(args.model_type)
                preprocessed_prompt_text = prepare_input(args, model, tokenizer, prompt_text)

                if model.__class__.__name__ in ["TransfoXLLMHeadModel"]:
                    tokenizer_kwargs = {"add_space_before_punct_symbol": True}
                else:
                    tokenizer_kwargs = {}

                encoded_prompt = tokenizer.encode(
                    preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", **tokenizer_kwargs
                )
            else:
                prefix = args.prefix if args.prefix else args.padding_text
                encoded_prompt = tokenizer.encode(prefix + prompt_text, add_special_tokens=False, return_tensors="pt")
            encoded_prompt = encoded_prompt.to(args.device)

            if encoded_prompt.size()[-1] == 0:
                input_ids = None
            else:
                input_ids = encoded_prompt

            output_sequences = model.generate(
                input_ids=input_ids,
                max_length=args.length + len(encoded_prompt[0]),
                temperature=args.temperature,
                top_k=args.k,
                top_p=args.p,
                repetition_penalty=args.repetition_penalty,
                do_sample=True,
                num_return_sequences=args.num_return_sequences,
            )

            # Remove the batch dimension when returning multiple sequences
            if len(output_sequences.shape) > 2:
                output_sequences.squeeze_()

            generated_sequences = []

            for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
                print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
                generated_sequence = generated_sequence.tolist()

                # Decode text
                text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

                # Remove all text after the stop token
                text = text[: text.find(args.stop_token) if args.stop_token else None]

                # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
                total_sequence = (
                    text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]#prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
                )

                generated_sequences.append(total_sequence)
                print(prompt_text + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
                print(total_sequence)
                print(reference)

                #references = [reference.split(" ")]
                #hypotheses = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :].split(" ")
                #references = " " + references
                references = [tokenizer.encode(references)]
                hypotheses = tokenizer.encode(text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :])
                bleus = [sentence_bleu(references, hypotheses, weights=(1, 0, 0, 0)), sentence_bleu(references, hypotheses, weights=(0, 1, 0, 0)), sentence_bleu(references, hypotheses, weights=(0, 0, 1, 0)), sentence_bleu(references, hypotheses, weights=(0, 0, 0, 1))]
                print("bleu:", bleus)
                for g in range(len(bleus)):
                    sum[g] += bleus[g]
                num += 1
                f.write(total_sequence + "\n")
        except:
            pass

    for g in range(len(bleus)):
        summ[g] = summ[g] / len(lines)
    print(summ)
    f.close()
    return generated_sequences


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--model_type MODEL_TYPE]
                             [--model_name_or_path MODEL_NAME_OR_PATH]
                             [--input_file INPUT_FILE] [--prompt PROMPT]
                             [--length LENGTH] [--stop_token STOP_TOKEN]
                             [--temperature TEMPERATURE]
                             [--repetition_penalty REPETITION_PENALTY] [--k K]
                             [--p P] [--prefix PREFIX]
                             [--padding_text PADDING_TEXT]
                             [--xlm_language XLM_LANGUAGE] [--seed SEED]
                             [--no_cuda]
                             [--num_return_sequences NUM_RETURN_SEQUENCES]
                             [--fp16]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-26e2174f-ac37-4877-917a-62af46de6598.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
dt["gens"] = gens.values()